### SET UP

In [13]:
import numpy as np
import pandas as pd
from data.DataManager import DataManager
from data.DataGenerator import DataGenerator
from data.DataCollectors import HistoricalDataCollector, HistoricalDataCollectorParquet
from main.RandomnessAnalysis import RandomnessAnalysis
from main.PredictableDayAnalysis import PredictableDayAnalysis
from utils.Analysis import get_assets_properties, localization_predictable_intervals
from utils.MultiTester import MultiTester
from utils.VisualizationTools import plot_block_frequencies
from utils.Analysis import intervals_analysis
from utils.VisualizationTools import plot_test

In [ ]:
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }
s = 2


In [3]:
year, month, day = 2025, 2, None

In [8]:
historical_collector = HistoricalDataCollectorParquet(pairs=asset_pairs, year=year, month=month, day=day)
historical_collector.collect()

[SYSTEM] Processing BTCUSDT...
[SYSTEM] Parquet data already available for BTCUSDT → data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet → Skipping download.
[SYSTEM] Processing ETHUSDT...
[SYSTEM] Parquet data already available for ETHUSDT → data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet → Skipping download.
[SYSTEM] Processing SOLUSDT...
[SYSTEM] Downloading SOLUSDT-trades-2025-02.zip from https://data.binance.vision/data/spot/monthly/trades/SOLUSDT/SOLUSDT-trades-2025-02.zip ...
[SYSTEM] File downloaded: SOLUSDT-trades-2025-02.zip
[SYSTEM] Extracting data/raw_data\SOLUSDT-trades-2025-02.zip ...
[SYSTEM] Extraction finished.
[SYSTEM] Converting SOLUSDT-trades-2025-02.csv → Parquet ...
[SYSTEM] Saved parquet to: data/raw_data_parquet\SOLUSDT-trades-2025-02.parquet
[SYSTEM] Deleted temporary CSV file: data/raw_data\SOLUSDT-trades-2025-02.csv
[SYSTEM] Deleted ZIP: data/raw_data\SOLUSDT-trades-2025-02.zip
[SYSTEM] Processing BNBUSDT...
[SYSTEM] Downloading BNBUSDT-trades-2025-02.z

In [8]:
import pandas as pd

pair = "BTCUSDT"
data_manager = DataManager(["BTCUSDT"], symbols, year=year, month=month, day=day, aggregation_level=1)
blocks = data_manager.block_constructor(block_size=2, overlapping=True)
# 1. Charger directement le parquet "raw_data_parquet"
parquet_file = f"data/raw_data_parquet/{pair}-trades-{year}-{month:02d}-{day:02d}.parquet"
df_parquet = pd.read_parquet(parquet_file)
print(f"[INFO] Fichier parquet brut : {parquet_file}")
print(f"[INFO] Nombre de lignes dans le parquet : {len(df_parquet)}")
print(f"[INFO] Colonnes : {df_parquet.columns.tolist()}")
print(df_parquet.head())

# 2. Récupérer le DataFrame préprocessé par DataManager
df_preprocessed = data_manager.datasets[pair]
print("\n[INFO] DataManager dataset :")
print(f"[INFO] Nombre de lignes après preprocessing : {len(df_preprocessed)}")
print(f"[INFO] Colonnes : {df_preprocessed.columns.tolist()}")
print(df_preprocessed.head())

# 3. Récupérer les blocs
blocks = data_manager.block_constructor(block_size=2, overlapping=True)
blocks_btc = blocks[pair]
print("\n[INFO] Blocs :")
print(f"[INFO] Nombre total de blocs : {len(blocks_btc)} (should be n_rows - block_size + 1)")
print(f"[INFO] Forme blocs : {blocks_btc.shape}")
print(blocks_btc.head(10))  # afficher 10 blocs


[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02-01.parquet
[INFO] Fichier parquet brut : data/raw_data_parquet/BTCUSDT-trades-2025-02-01.parquet
[INFO] Nombre de lignes dans le parquet : 2588786
[INFO] Colonnes : ['trade_id', 'price', 'volume', 'quote_qty', 'timestamp', 'is_buyer_maker', 'is_best_match']
     trade_id      price   volume  quote_qty         timestamp  \
0  4495881901  102429.56  0.00005   5.121478  1738368000182381   
1  4495881902  102429.56  0.00006   6.145774  1738368000182381   
2  4495881903  102429.56  0.00006   6.145774  1738368000302865   
3  4495881904  102429.56  0.00006   6.145774  1738368000302865   
4  4495881905  102429.56  0.00012  12.291547  1738368000302865   

   is_buyer_maker  is_best_match  
0           False           True  
1           False           True  
2           False           True  
3           False           True  
4           False           True  

[INFO] DataManager dataset :
[INFO] Nombre de lignes 

### Efficient vs Inefficient days

#### BTC

In [9]:
#Nouvelle version : 
data_manager = DataManager(["BTCUSDT"], symbols, year=year, month=month, day=day, aggregation_level=1)
blocks = data_manager.block_constructor(block_size=2, overlapping=True)
analysis = PredictableDayAnalysis(pair="BTCUSDT", data_manager=data_manager)
analysis.analyze_days(block_size=2, n_jobs=8)

analysis.efficient_df

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[SYSTEM] Starting analysis → 28 days to process...


[BTCUSDT] Analyzing days: 100%|██████████| 28/28 [21:06<00:00, 45.23s/it]

[SYSTEM] Analysis completed : 28 days analyzed → 0 efficient, 28 inefficient.


""


In [10]:
analysis.inefficient_df

,Jump fraction,Autocorrelation,Autocorrelation of squared returns,Student distribution degree of freedom,Student distribution mean,Student distribution standard deviation,Returns mean,Fractions of zero-returns,KL Divergence statistics,Empirical quantile,Hypothesis
0,0.009263,-0.036351,0.200766,1.988286,-1.284749e-08,8.283958e-07,-1.480791e-09,0.0,6.623435e+04,6.634897,True
1,0.009993,-0.126350,0.306825,0.543643,-1.157639e-08,1.904304e-07,-3.173761e-08,0.0,8.223162e+04,6.634897,True
2,0.008610,-0.279057,0.314318,1.988331,-8.971039e-09,1.152788e-06,3.268467e-10,0.0,1.876580e+05,6.634897,True
3,0.007354,-0.104142,0.203054,1.988306,-3.746196e-08,1.492806e-06,-9.477338e-09,0.0,8.055670e+05,6.634897,True
4,0.008015,-0.189714,0.389228,1.988325,5.129374e-10,1.574108e-06,-3.826893e-10,0.0,7.867104e+05,6.634897,True
5,0.007836,-0.120107,0.167781,1.988297,-1.632158e-08,1.327490e-06,1.171574e-08,0.0,4.160548e+05,6.634897,True
6,0.007812,-0.074027,0.250404,0.527825,-7.215345e-09,3.215333e-07,-5.093478e-10,0.0,7.226034e+05,6.634897,True
7,0.008217,-0.121779,0.410299,1.988287,-1.251510e-08,1.099314e-06,-2.233954e-08,0.0,3.146635e+05,6.634897,True
8,0.010853,-0.046108,0.417502,0.562942,-4.823422e-09,1.929159e-07,2.165876e-09,0.0,4.190663e+04,6.634897,True
9,0.011210,-0.093456,0.425607,0.556852,-1.533488e-08,1.982305e-07,-4.727323e-08,0.0,4.611823e+04,6.634897,True


#### ETH

In [11]:
data_manager = DataManager(["ETHUSDT"], symbols, year=year, month=month, aggregation_level=1)
blocks = data_manager.block_constructor(block_size=2, overlapping=True)
analysis = PredictableDayAnalysis(pair="ETHUSDT",data_manager=data_manager)
analysis.analyze_days(block_size=2, n_jobs=8)
analysis.efficient_df

[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[SYSTEM] Starting analysis → 28 days to process...


[ETHUSDT] Analyzing days: 100%|██████████| 28/28 [17:28<00:00, 37.46s/it]  

[SYSTEM] Analysis completed : 28 days analyzed → 0 efficient, 28 inefficient.


""


In [12]:
analysis.inefficient_df

,Jump fraction,Autocorrelation,Autocorrelation of squared returns,Student distribution degree of freedom,Student distribution mean,Student distribution standard deviation,Returns mean,Fractions of zero-returns,KL Divergence statistics,Empirical quantile,Hypothesis
0,0.004087,-0.074587,0.348325,1.987908,1.129815e-06,0.000005,6.185844e-09,0.0,3.876362e+05,6.634897,True
1,0.003829,-0.058151,0.305684,1.988129,5.600608e-07,0.000005,-8.038639e-08,0.0,4.032643e+05,6.634897,True
2,0.004281,-0.245941,0.306935,1.988061,7.486087e-07,0.000006,-2.952445e-09,0.0,4.263340e+05,6.634897,True
3,0.004412,-0.128034,0.451686,2.218613,9.334083e-07,0.000006,-2.090363e-08,0.0,7.950268e+05,6.634897,True
4,0.004026,-0.183074,0.252192,1.988079,6.827948e-07,0.000006,-4.827831e-08,0.0,1.016059e+06,6.634897,True
5,0.003818,-0.085103,0.336550,2.604725,7.746969e-07,0.000006,1.708692e-08,0.0,4.916025e+05,6.634897,True
6,0.003614,0.015342,0.341894,2.637785,2.240145e-07,0.000006,-3.008222e-08,0.0,9.150241e+05,6.634897,True
7,0.004307,-0.200863,0.388313,2.385467,5.263268e-07,0.000006,1.541637e-08,0.0,1.024360e+06,6.634897,True
8,0.005086,-0.173044,0.357468,1.988006,8.299188e-07,0.000007,-2.700892e-08,0.0,1.351127e+06,6.634897,True
9,0.003482,-0.090623,0.245380,1.988203,3.610544e-07,0.000005,-3.427254e-08,0.0,1.788815e+05,6.634897,True


### Predictable intervals locations (BTC)

In [18]:
df = localization_predictable_intervals(data_manager, "ETHUSDT", test='NP Statistic')
df.head(10)

,Timestamp Start,Timestamp End,Test Stat,Quantile 99%,P-value,Hypothesis
Rank,,,,,,
1,2990229,3022381,657152.336909,2331.348021,0.0,True
2,3247453,3279605,655272.294246,2331.348021,0.0,True
3,3279606,3311758,655112.137552,2331.348021,0.0,True
4,3376065,3408217,654037.352028,2331.348021,0.0,True
5,3215300,3247452,654016.941736,2331.348021,0.0,True
6,3311759,3343911,653794.889422,2331.348021,0.0,True
7,3022382,3054534,653316.465348,2331.348021,0.0,True
8,2861617,2893769,652765.998939,2331.348021,0.0,True
9,3086688,3118840,652640.955683,2331.348021,0.0,True


### Aggregation level 1

In [7]:
data_manager = DataManager(asset_pairs, symbols, year=year, month=month, aggregation_level=1)

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BNBUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\AVAXUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\UNIUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\LINKUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\AXSUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\RENDERUSDT-trades-2025-02.parquet


#### Non overlapping

In [8]:
blocks_size_2 = data_manager.block_constructor(block_size=2, overlapping=False)
blocks_size_3 = data_manager.block_constructor(block_size=3, overlapping=False)
blocks_size_5 = data_manager.block_constructor(block_size=5, overlapping=False)

##### Block size 2

In [21]:
blocks_btc = blocks_size_2['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [22]:
test_entropy = analyser.entropy_bias_test()
test_entropy

,Entropy Bias test
Bias,9191363.023922
Quantile 90%,6.251389
Quantile 95%,7.814728
Quantile 99%,11.344867
P-value,0.0
Mean,3
Hypothesis 1,True


In [23]:
blocks_eth = blocks_size_2['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [24]:
test_entropy = analyser.entropy_bias_test()
test_entropy

,Entropy Bias test
Bias,10017123.741588
Quantile 90%,6.251389
Quantile 95%,7.814728
Quantile 99%,11.344867
P-value,0.0
Mean,3
Hypothesis 1,True


In [25]:
blocks_uni = blocks_size_2['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [26]:
test_entropy = analyser.entropy_bias_test()
test_entropy

,Entropy Bias test
Bias,150575.791039
Quantile 90%,6.251389
Quantile 95%,7.814728
Quantile 99%,11.344867
P-value,0.0
Mean,3
Hypothesis 1,True


##### Block size 3

In [27]:
blocks_btc = blocks_size_3['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [28]:
blocks_eth = blocks_size_3['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [29]:
blocks_uni = blocks_size_3['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

##### Block size 5

In [30]:
blocks_btc = blocks_size_5['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [32]:
blocks_eth = blocks_size_5['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [33]:
blocks_uni = blocks_size_5['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

#### Overlapping

In [9]:
blocks_size_2 = data_manager.block_constructor(block_size=2, overlapping=True)
blocks_size_3 = data_manager.block_constructor(block_size=3, overlapping=True)
blocks_size_5 = data_manager.block_constructor(block_size=5, overlapping=True)

##### Block size 2

In [35]:
blocks_btc = blocks_size_2['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [36]:
test_divergence = analyser.KL_divergence_test()
test_divergence

,NP Statistic test
NP Statistic,18368358.279773
Quantile 90%,2.705543
Quantile 95%,3.841459
Quantile 99%,6.634897
P-value,0.0
Mean,1
Hypothesis 1,True


In [10]:
blocks_btc = blocks_size_2['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

test_divergence = analyser.KL_divergence_test()
test_divergence

,NP Statistic test
NP Statistic,18368358.279773
Quantile 90%,2.705543
Quantile 95%,3.841459
Quantile 99%,6.634897
P-value,0.0
Mean,1
Hypothesis 1,True


In [37]:
blocks_eth = blocks_size_2['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [38]:
test_divergence = analyser.KL_divergence_test()
test_divergence

,NP Statistic test
NP Statistic,19672444.320302
Quantile 90%,2.705543
Quantile 95%,3.841459
Quantile 99%,6.634897
P-value,0.0
Mean,1
Hypothesis 1,True


In [39]:
blocks_uni = blocks_size_2['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [40]:
test_divergence = analyser.KL_divergence_test()
test_divergence

,NP Statistic test
NP Statistic,302596.762078
Quantile 90%,2.705543
Quantile 95%,3.841459
Quantile 99%,6.634897
P-value,0.0
Mean,1
Hypothesis 1,True


##### Block size 3

In [41]:
blocks_btc = blocks_size_3['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [42]:
blocks_eth = blocks_size_3['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [43]:
blocks_uni = blocks_size_3['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

##### Block size 5

In [44]:
blocks_btc = blocks_size_5['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [45]:
blocks_eth = blocks_size_5['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [46]:
blocks_uni = blocks_size_5['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [16]:
for name in dir():
    if not name.startswith('_') and name not in ['important_var1', 'important_var2']:
        del globals()[name]
        import numpy as np
import pandas as pd
from data.DataManager import DataManager
from data.DataGenerator import DataGenerator
from data.DataCollectors import HistoricalDataCollector, HistoricalDataCollectorParquet
from main.RandomnessAnalysis import RandomnessAnalysis
from main.PredictableDayAnalysis import PredictableDayAnalysis
from utils.Analysis import get_assets_properties, localization_predictable_intervals
from utils.MultiTester import MultiTester
from utils.VisualizationTools import plot_block_frequencies
from utils.Analysis import intervals_analysis
from utils.VisualizationTools import plot_test
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }
s = 2
year, month, day = 2025, 2, None

### Aggregation level 5

In [17]:
data_manager = DataManager(asset_pairs, symbols, year=year, month=month, aggregation_level=5)

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BNBUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\AVAXUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\UNIUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\LINKUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\AXSUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\RENDERUSDT-trades-2025-02.parquet


#### Non overlapping

In [18]:
blocks_size_2 = data_manager.block_constructor(block_size=2, overlapping=False)
blocks_size_3 = data_manager.block_constructor(block_size=3, overlapping=False)
blocks_size_5 = data_manager.block_constructor(block_size=5, overlapping=False)

##### Block size 2

In [19]:
blocks_btc = blocks_size_2['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [20]:
test_entropy = analyser.entropy_bias_test()
test_entropy

,Entropy Bias test
Bias,7472327.945792
Quantile 90%,6.251389
Quantile 95%,7.814728
Quantile 99%,11.344867
P-value,0.0
Mean,3
Hypothesis 1,True


In [21]:
blocks_eth = blocks_size_2['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [22]:
test_entropy = analyser.entropy_bias_test()
test_entropy

,Entropy Bias test
Bias,6593009.990946
Quantile 90%,6.251389
Quantile 95%,7.814728
Quantile 99%,11.344867
P-value,0.0
Mean,3
Hypothesis 1,True


In [23]:
blocks_uni = blocks_size_2['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [24]:
test_entropy = analyser.entropy_bias_test()
test_entropy

,Entropy Bias test
Bias,112389.321366
Quantile 90%,6.251389
Quantile 95%,7.814728
Quantile 99%,11.344867
P-value,0.0
Mean,3
Hypothesis 1,True


##### Block size 3

In [25]:
blocks_btc = blocks_size_3['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [26]:
blocks_eth = blocks_size_3['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [27]:
blocks_uni = blocks_size_3['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

##### Block size 5

In [28]:
blocks_btc = blocks_size_5['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [29]:
blocks_eth = blocks_size_5['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [30]:
blocks_uni = blocks_size_5['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

#### Overlapping

In [31]:
blocks_size_2 = data_manager.block_constructor(block_size=2, overlapping=True)
blocks_size_3 = data_manager.block_constructor(block_size=3, overlapping=True)
blocks_size_5 = data_manager.block_constructor(block_size=5, overlapping=True)

##### Block size 2

In [32]:
blocks_btc = blocks_size_2['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [33]:
test_divergence = analyser.KL_divergence_test()
test_divergence

,NP Statistic test
NP Statistic,14934056.580334
Quantile 90%,2.705543
Quantile 95%,3.841459
Quantile 99%,6.634897
P-value,0.0
Mean,1
Hypothesis 1,True


In [34]:
blocks_eth = blocks_size_2['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [35]:
test_divergence = analyser.KL_divergence_test()
test_divergence

,NP Statistic test
NP Statistic,12922702.487924
Quantile 90%,2.705543
Quantile 95%,3.841459
Quantile 99%,6.634897
P-value,0.0
Mean,1
Hypothesis 1,True


In [36]:
blocks_uni = blocks_size_2['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [37]:
test_divergence = analyser.KL_divergence_test()
test_divergence

,NP Statistic test
NP Statistic,224159.751936
Quantile 90%,2.705543
Quantile 95%,3.841459
Quantile 99%,6.634897
P-value,0.0
Mean,1
Hypothesis 1,True


##### Block size 3

In [38]:
blocks_btc = blocks_size_3['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [39]:
blocks_eth = blocks_size_3['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [40]:
blocks_uni = blocks_size_3['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

##### Block size 5

In [41]:
blocks_btc = blocks_size_5['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [42]:
blocks_eth = blocks_size_5['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [43]:
blocks_uni = blocks_size_5['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [44]:
for name in dir():
    if not name.startswith('_') and name not in ['important_var1', 'important_var2']:
        del globals()[name]
import numpy as np
import pandas as pd
from data.DataManager import DataManager
from data.DataGenerator import DataGenerator
from data.DataCollectors import HistoricalDataCollector, HistoricalDataCollectorParquet
from main.RandomnessAnalysis import RandomnessAnalysis
from main.PredictableDayAnalysis import PredictableDayAnalysis
from utils.Analysis import get_assets_properties, localization_predictable_intervals
from utils.MultiTester import MultiTester
from utils.VisualizationTools import plot_block_frequencies
from utils.Analysis import intervals_analysis
from utils.VisualizationTools import plot_test
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }
s = 2
year, month, day = 2025, 2, None

### Aggregation level 20

In [45]:
data_manager = DataManager(asset_pairs, symbols, year=year, month=month, aggregation_level=20)

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BNBUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\AVAXUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\UNIUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\LINKUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\AXSUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\RENDERUSDT-trades-2025-02.parquet


#### Non overlapping

In [46]:
blocks_size_2 = data_manager.block_constructor(block_size=2, overlapping=False)
blocks_size_3 = data_manager.block_constructor(block_size=3, overlapping=False)
blocks_size_5 = data_manager.block_constructor(block_size=5, overlapping=False)

##### Block size 2

In [47]:
blocks_btc = blocks_size_2['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [48]:
test_entropy = analyser.entropy_bias_test()
test_entropy

,Entropy Bias test
Bias,2328722.680654
Quantile 90%,6.251389
Quantile 95%,7.814728
Quantile 99%,11.344867
P-value,0.0
Mean,3
Hypothesis 1,True


In [49]:
blocks_eth = blocks_size_2['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [50]:
test_entropy = analyser.entropy_bias_test()
test_entropy

,Entropy Bias test
Bias,1422084.506984
Quantile 90%,6.251389
Quantile 95%,7.814728
Quantile 99%,11.344867
P-value,0.0
Mean,3
Hypothesis 1,True


In [51]:
blocks_uni = blocks_size_2['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [52]:
test_entropy = analyser.entropy_bias_test()
test_entropy

,Entropy Bias test
Bias,8507.952062
Quantile 90%,6.251389
Quantile 95%,7.814728
Quantile 99%,11.344867
P-value,0.0
Mean,3
Hypothesis 1,True


##### Block size 3

In [53]:
blocks_btc = blocks_size_3['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [54]:
blocks_eth = blocks_size_3['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [55]:
blocks_uni = blocks_size_3['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

##### Block size 5

In [56]:
blocks_btc = blocks_size_5['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [57]:
blocks_eth = blocks_size_5['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [58]:
blocks_uni = blocks_size_5['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

#### Overlapping

In [59]:
blocks_size_2 = data_manager.block_constructor(block_size=2, overlapping=True)
blocks_size_3 = data_manager.block_constructor(block_size=3, overlapping=True)
blocks_size_5 = data_manager.block_constructor(block_size=5, overlapping=True)

##### Block size 2

In [60]:
blocks_btc = blocks_size_2['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [61]:
test_divergence = analyser.KL_divergence_test()
test_divergence

,NP Statistic test
NP Statistic,4650518.801562
Quantile 90%,2.705543
Quantile 95%,3.841459
Quantile 99%,6.634897
P-value,0.0
Mean,1
Hypothesis 1,True


In [62]:
blocks_eth = blocks_size_2['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [63]:
test_divergence = analyser.KL_divergence_test()
test_divergence

,NP Statistic test
NP Statistic,2800524.632054
Quantile 90%,2.705543
Quantile 95%,3.841459
Quantile 99%,6.634897
P-value,0.0
Mean,1
Hypothesis 1,True


In [64]:
blocks_uni = blocks_size_2['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [65]:
test_divergence = analyser.KL_divergence_test()
test_divergence

,NP Statistic test
NP Statistic,17248.234977
Quantile 90%,2.705543
Quantile 95%,3.841459
Quantile 99%,6.634897
P-value,0.0
Mean,1
Hypothesis 1,True


##### Block size 3

In [66]:
blocks_btc = blocks_size_3['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [67]:
blocks_eth = blocks_size_3['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [68]:
blocks_uni = blocks_size_3['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

##### Block size 5

In [69]:
blocks_btc = blocks_size_5['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [70]:
blocks_eth = blocks_size_5['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [71]:
blocks_uni = blocks_size_5['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [72]:
for name in dir():
    if not name.startswith('_') and name not in ['important_var1', 'important_var2']:
        del globals()[name]
import numpy as np
import pandas as pd
from data.DataManager import DataManager
from data.DataGenerator import DataGenerator
from data.DataCollectors import HistoricalDataCollector, HistoricalDataCollectorParquet
from main.RandomnessAnalysis import RandomnessAnalysis
from main.PredictableDayAnalysis import PredictableDayAnalysis
from utils.Analysis import get_assets_properties, localization_predictable_intervals
from utils.MultiTester import MultiTester
from utils.VisualizationTools import plot_block_frequencies
from utils.Analysis import intervals_analysis
from utils.VisualizationTools import plot_test
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }
s = 2
year, month, day = 2025, 2, None

### Aggregation level 50

In [73]:
data_manager = DataManager(asset_pairs, symbols, year=year, month=month, aggregation_level=50)

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BNBUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\AVAXUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\UNIUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\LINKUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\AXSUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\RENDERUSDT-trades-2025-02.parquet


#### Non overlapping

In [74]:
blocks_size_2 = data_manager.block_constructor(block_size=2, overlapping=False)
blocks_size_3 = data_manager.block_constructor(block_size=3, overlapping=False)
blocks_size_5 = data_manager.block_constructor(block_size=5, overlapping=False)

##### Block size 2

In [75]:
blocks_btc = blocks_size_2['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [76]:
test_entropy = analyser.entropy_bias_test()
test_entropy

,Entropy Bias test
Bias,794338.262043
Quantile 90%,6.251389
Quantile 95%,7.814728
Quantile 99%,11.344867
P-value,0.0
Mean,3
Hypothesis 1,True


In [77]:
blocks_eth = blocks_size_2['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [78]:
test_entropy = analyser.entropy_bias_test()
test_entropy

,Entropy Bias test
Bias,282532.587013
Quantile 90%,6.251389
Quantile 95%,7.814728
Quantile 99%,11.344867
P-value,0.0
Mean,3
Hypothesis 1,True


In [79]:
blocks_uni = blocks_size_2['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [80]:
test_entropy = analyser.entropy_bias_test()
test_entropy

,Entropy Bias test
Bias,2130.577367
Quantile 90%,6.251389
Quantile 95%,7.814728
Quantile 99%,11.344867
P-value,0.0
Mean,3
Hypothesis 1,True


##### Block size 3

In [81]:
blocks_btc = blocks_size_3['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [82]:
blocks_eth = blocks_size_3['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [83]:
blocks_uni = blocks_size_3['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

##### Block size 5

In [84]:
blocks_btc = blocks_size_5['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [85]:
blocks_eth = blocks_size_5['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [86]:
blocks_uni = blocks_size_5['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

#### Overlapping

In [87]:
blocks_size_2 = data_manager.block_constructor(block_size=2, overlapping=False)
blocks_size_3 = data_manager.block_constructor(block_size=3, overlapping=False)
blocks_size_5 = data_manager.block_constructor(block_size=5, overlapping=False)

##### Block size 2

In [88]:
blocks_btc = blocks_size_2['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [89]:
test_divergence = analyser.KL_divergence_test()
test_divergence

,NP Statistic test
NP Statistic,794075.749332
Quantile 90%,2.705543
Quantile 95%,3.841459
Quantile 99%,6.634897
P-value,0.0
Mean,1
Hypothesis 1,True


In [90]:
blocks_eth = blocks_size_2['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [91]:
test_divergence = analyser.KL_divergence_test()
test_divergence

,NP Statistic test
NP Statistic,277986.759315
Quantile 90%,2.705543
Quantile 95%,3.841459
Quantile 99%,6.634897
P-value,0.0
Mean,1
Hypothesis 1,True


In [92]:
blocks_uni = blocks_size_2['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [93]:
test_divergence = analyser.KL_divergence_test()
test_divergence

,NP Statistic test
NP Statistic,2127.876568
Quantile 90%,2.705543
Quantile 95%,3.841459
Quantile 99%,6.634897
P-value,0.0
Mean,1
Hypothesis 1,True


##### Block size 3

In [94]:
blocks_btc = blocks_size_3['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [95]:
blocks_eth = blocks_size_3['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [96]:
blocks_uni = blocks_size_3['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

##### Block size 5

In [97]:
blocks_btc = blocks_size_5['BTCUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_btc, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [98]:
blocks_eth = blocks_size_5['ETHUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_eth, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [99]:
blocks_uni = blocks_size_5['UNIUSDT']
analyser = RandomnessAnalysis(blocks_df=blocks_uni, s=s)
frequencies_df = analyser.compute_blocks_frequencies()
plot_block_frequencies(frequencies_df)

In [100]:
for name in dir():
    if not name.startswith('_') and name not in ['important_var1', 'important_var2']:
        del globals()[name]
import numpy as np
import pandas as pd
from data.DataManager import DataManager
from data.DataGenerator import DataGenerator
from data.DataCollectors import HistoricalDataCollector, HistoricalDataCollectorParquet
from main.RandomnessAnalysis import RandomnessAnalysis
from main.PredictableDayAnalysis import PredictableDayAnalysis
from utils.Analysis import get_assets_properties, localization_predictable_intervals
from utils.MultiTester import MultiTester
from utils.VisualizationTools import plot_block_frequencies
from utils.Analysis import intervals_analysis
from utils.VisualizationTools import plot_test
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }
s = 2
year, month, day = 2025, 2, None

### MultiTester BTC

In [101]:
multi_tester = MultiTester(asset='BTCUSDT',symbols=symbols,overlapping=False)
df_test_block = multi_tester.test_by_block_size(test='Entropy Bias',
                                    max_block_size=15,
                                    year=year,
                                    month=month,
                                    aggregation_level=5)
df_test_block

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet


,Test statistic,Quantile 99,Quantile 95,Quantile 90,Mean
Block size,,,,,
1,3.432793e+03,6.634897,3.841459,2.705543,1
2,7.472328e+06,11.344867,7.814728,6.251389,3
3,1.079882e+07,18.475307,14.067140,12.017037,7
4,1.263876e+07,30.577914,24.995790,22.307130,15
5,1.376251e+07,52.191395,44.985343,41.421736,31
6,1.451934e+07,92.010024,82.528727,77.745385,63
7,1.505986e+07,166.987390,154.301516,147.804813,127
8,1.547250e+07,310.457388,293.247835,284.335908,255
9,1.579451e+07,588.297794,564.696133,552.373933,511


In [102]:
plot_test(x_values=df_test_block.index, 
          y1_values=df_test_block['Test statistic'].values,
          y2_values=df_test_block['Quantile 99'].values,
          test='Entropy Bias',
          x_label='Block size',
          pair='BTCUSDT')

In [1]:
for name in dir():
    if not name.startswith('_') and name not in ['important_var1', 'important_var2']:
        del globals()[name]
import numpy as np
import pandas as pd
from data.DataManager import DataManager
from data.DataGenerator import DataGenerator
from data.DataCollectors import HistoricalDataCollector, HistoricalDataCollectorParquet
from main.RandomnessAnalysis import RandomnessAnalysis
from main.PredictableDayAnalysis import PredictableDayAnalysis
from utils.Analysis import get_assets_properties, localization_predictable_intervals
from utils.MultiTester import MultiTester
from utils.VisualizationTools import plot_block_frequencies
from utils.Analysis import intervals_analysis
from utils.VisualizationTools import plot_test
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }
s = 2
year, month, day = 2025, 2, None

In [2]:
multi_tester = MultiTester(asset='BTCUSDT',symbols=symbols,overlapping=True)
df_test_agg = multi_tester.test_by_aggregation_level(test='NP Statistic',
                                           max_aggregation_level=50,
                                           year=year,
                                           month=month,
                                           block_size=2,
                                           step =2 )
df_test_agg

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading

,Test statistic,Quantile 90,Quantile 95,Quantile 99,Mean
Aggregation level,,,,,
1,1.836836e+07,2.705543,3.841459,6.634897,1
3,2.166955e+07,2.705543,3.841459,6.634897,1
5,1.493406e+07,2.705543,3.841459,6.634897,1
7,1.154675e+07,2.705543,3.841459,6.634897,1
9,9.427231e+06,2.705543,3.841459,6.634897,1
11,7.982576e+06,2.705543,3.841459,6.634897,1
13,6.915830e+06,2.705543,3.841459,6.634897,1
15,6.092276e+06,2.705543,3.841459,6.634897,1
17,5.430373e+06,2.705543,3.841459,6.634897,1


In [3]:
plot_test(x_values=df_test_agg.index, 
          y1_values=df_test_agg['Test statistic'].values,
          y2_values=df_test_agg['Quantile 99'].values,
          test='NP Statistic',
          x_label='Aggregation level',
          pair='BTCUSDT')

In [2]:
btc_multi_tester = MultiTester(asset='BTCUSDT',symbols=symbols,overlapping=False)
btc_multi_tester.plot_3D_test_result(asset='BTCUSDT',
                                     test='Entropy Bias',
                                     max_block_size=15,
                                     year=year,month=month,
                                     max_aggregation_level=50,
                                     step_block=2,
                                     step_aggregation =2)

[BTCUSDT] Grid test 3D:   0%|          | 0/200 [00:00<?, ?it/s]

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet


[BTCUSDT] Grid test 3D:   0%|          | 1/200 [00:55<3:03:46, 55.41s/it]

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet


[BTCUSDT] Grid test 3D:   1%|          | 2/200 [01:24<2:11:33, 39.87s/it]

...

[BTCUSDT] Grid test 3D:  99%|█████████▉| 198/200 [1:42:05<01:00, 30.14s/it]

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet


[BTCUSDT] Grid test 3D: 100%|█████████▉| 199/200 [1:42:37<00:30, 30.53s/it]

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet


[BTCUSDT] Grid test 3D: 100%|██████████| 200/200 [1:43:08<00:00, 30.94s/it]


In [6]:
for name in dir():
    if not name.startswith('_') and name not in ['important_var1', 'important_var2']:
        del globals()[name]
import numpy as np
import pandas as pd
from data.DataManager import DataManager
from data.DataGenerator import DataGenerator
from data.DataCollectors import HistoricalDataCollector, HistoricalDataCollectorParquet
from main.RandomnessAnalysis import RandomnessAnalysis
from main.PredictableDayAnalysis import PredictableDayAnalysis
from utils.Analysis import get_assets_properties, localization_predictable_intervals
from utils.MultiTester import MultiTester
from utils.VisualizationTools import plot_block_frequencies
from utils.Analysis import intervals_analysis
from utils.VisualizationTools import plot_test
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }
s = 2
year, month, day = 2025, 2, None


### MultiTester ETH

In [5]:
multi_tester = MultiTester(asset='ETHUSDT',symbols=symbols,overlapping=False)
df_test_block = multi_tester.test_by_block_size(test='NP Statistic',
                                    max_block_size=15,
                                    year=year,
                                    month=month,
                                    aggregation_level=5)
df_test_block

[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet


ValueError: No group keys passed!

In [ ]:
plot_test(x_values=df_test_block.index, 
          y1_values=df_test_block['Test statistic'].values,
          y2_values=df_test_block['Quantile 99'].values,
          test='Entropy Bias',
          x_label='Block size',
          pair='BTCUSDT')

In [ ]:
for name in dir():
    if not name.startswith('_') and name not in ['important_var1', 'important_var2']:
        del globals()[name]
import numpy as np
import pandas as pd
from data.DataManager import DataManager
from data.DataGenerator import DataGenerator
from data.DataCollectors import HistoricalDataCollector, HistoricalDataCollectorParquet
from main.RandomnessAnalysis import RandomnessAnalysis
from main.PredictableDayAnalysis import PredictableDayAnalysis
from utils.Analysis import get_assets_properties, localization_predictable_intervals
from utils.MultiTester import MultiTester
from utils.VisualizationTools import plot_block_frequencies
from utils.Analysis import intervals_analysis
from utils.VisualizationTools import plot_test
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }
s = 2
year, month, day = 2025, 2, None

In [ ]:
multi_tester = MultiTester(asset='BTCUSDT',symbols=symbols,overlapping=True)
df_test_agg = multi_tester.test_by_aggregation_level(test='NP Statistic',
                                           max_aggregation_level=50,
                                           year=year,
                                           month=month,
                                           block_size=2)
df_test_agg

In [ ]:
plot_test(x_values=df_test_agg.index, 
          y1_values=df_test_agg['Test statistic'].values,
          y2_values=df_test_agg['Quantile 99'].values,
          test='NP Statistic',
          x_label='Aggregation level',
          pair='BTCUSDT')

In [ ]:
for name in dir():
    if not name.startswith('_') and name not in ['important_var1', 'important_var2']:
        del globals()[name]
import numpy as np
import pandas as pd
from data.DataManager import DataManager
from data.DataGenerator import DataGenerator
from data.DataCollectors import HistoricalDataCollector, HistoricalDataCollectorParquet
from main.RandomnessAnalysis import RandomnessAnalysis
from main.PredictableDayAnalysis import PredictableDayAnalysis
from utils.Analysis import get_assets_properties, localization_predictable_intervals
from utils.MultiTester import MultiTester
from utils.VisualizationTools import plot_block_frequencies
from utils.Analysis import intervals_analysis
from utils.VisualizationTools import plot_test
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }
s = 2
year, month, day = 2025, 2, None

### MultiTester UNI

In [ ]:
multi_tester = MultiTester(asset='UNIUSDT',symbols=symbols,overlapping=False)
df_test_block = multi_tester.test_by_block_size(test='Entropy Bias',
                                    max_block_size=15,
                                    year=year,
                                    month=month,
                                    aggregation_level=5)
df_test_block

In [ ]:
plot_test(x_values=df_test_block.index, 
          y1_values=df_test_block['Test statistic'].values,
          y2_values=df_test_block['Quantile 99'].values,
          test='Entropy Bias',
          x_label='Block size',
          pair='BTCUSDT')

In [ ]:
for name in dir():
    if not name.startswith('_') and name not in ['important_var1', 'important_var2']:
        del globals()[name]
import numpy as np
import pandas as pd
from data.DataManager import DataManager
from data.DataGenerator import DataGenerator
from data.DataCollectors import HistoricalDataCollector, HistoricalDataCollectorParquet
from main.RandomnessAnalysis import RandomnessAnalysis
from main.PredictableDayAnalysis import PredictableDayAnalysis
from utils.Analysis import get_assets_properties, localization_predictable_intervals
from utils.MultiTester import MultiTester
from utils.VisualizationTools import plot_block_frequencies
from utils.Analysis import intervals_analysis
from utils.VisualizationTools import plot_test
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }
s = 2
year, month, day = 2025, 2, None

In [ ]:
multi_tester = MultiTester(asset='BTCUSDT',symbols=symbols,overlapping=True)
df_test_agg = multi_tester.test_by_aggregation_level(test='NP Statistic',
                                           max_aggregation_level=50,
                                           year=year,
                                           month=month,
                                           block_size=2)
df_test_agg

In [ ]:
plot_test(x_values=df_test_agg.index, 
          y1_values=df_test_agg['Test statistic'].values,
          y2_values=df_test_agg['Quantile 99'].values,
          test='NP Statistic',
          x_label='Aggregation level',
          pair='BTCUSDT')

In [ ]:
for name in dir():
    if not name.startswith('_') and name not in ['important_var1', 'important_var2']:
        del globals()[name]

import numpy as np
import pandas as pd
from data.DataManager import DataManager
from data.DataGenerator import DataGenerator
from data.DataCollectors import HistoricalDataCollector, HistoricalDataCollectorParquet
from main.RandomnessAnalysis import RandomnessAnalysis
from main.PredictableDayAnalysis import PredictableDayAnalysis
from utils.Analysis import get_assets_properties, localization_predictable_intervals
from utils.MultiTester import MultiTester
from utils.VisualizationTools import plot_block_frequencies
from utils.Analysis import intervals_analysis
from utils.VisualizationTools import plot_test
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }
s = 2
year, month, day = 2025, 2, None

### Fraction of predictable days

In [ ]:
historical_collector = HistoricalDataCollectorParquet(pairs=asset_pairs, year=year, month=month, day=day)
historical_collector.collect()

In [ ]:
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']

In [ ]:
intervals_analysis(pairs=asset_pairs,
                   symbols=symbols,
                   max_aggregation_level=50,
                   year=year,
                   month=[month])

In [ ]:
for name in dir():
    if not name.startswith('_') and name not in ['important_var1', 'important_var2']:
        del globals()[name]
import numpy as np
import pandas as pd
from data.DataManager import DataManager
from data.DataGenerator import DataGenerator
from data.DataCollectors import HistoricalDataCollector, HistoricalDataCollectorParquet
from main.RandomnessAnalysis import RandomnessAnalysis
from main.PredictableDayAnalysis import PredictableDayAnalysis
from utils.Analysis import get_assets_properties, localization_predictable_intervals
from utils.MultiTester import MultiTester
from utils.VisualizationTools import plot_block_frequencies
from utils.Analysis import intervals_analysis
from utils.VisualizationTools import plot_test
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }
s = 2
year, month, day = 2025, 2, None

In [ ]:
intervals_analysis(pairs=asset_pairs,
                   symbols=symbols,
                   max_aggregation_level=50,
                   year=year,
                   month=[month],
                   test = 'NP Statistic')

In [1]:
for name in dir():
    if not name.startswith('_') and name not in ['important_var1', 'important_var2']:
        del globals()[name]
    
import numpy as np
import pandas as pd
from data.DataManager import DataManager
from data.DataGenerator import DataGenerator
from data.DataCollectors import HistoricalDataCollector, HistoricalDataCollectorParquet
from main.RandomnessAnalysis import RandomnessAnalysis
from main.PredictableDayAnalysis import PredictableDayAnalysis
from utils.Analysis import get_assets_properties, localization_predictable_intervals
from utils.MultiTester import MultiTester
from utils.VisualizationTools import plot_block_frequencies
from utils.Analysis import intervals_analysis
from utils.VisualizationTools import plot_test
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }
s = 2
year, month, day = 2025, 2, None

### Assets properties

In [2]:
data_manager = DataManager(asset_pairs, symbols, year=year, month=month, aggregation_level=1)
df_prop = get_assets_properties(asset_pairs, s=2, year=year, month=month)
df_prop

[DataManager] Loading parquet file: data/raw_data_parquet\BTCUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\ETHUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\SOLUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\BNBUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\AVAXUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\UNIUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\LINKUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\AXSUSDT-trades-2025-02.parquet
[DataManager] Loading parquet file: data/raw_data_parquet\RENDERUSDT-trades-2025-02.parquet


,Mean price,Price standard deviation,Mean return,Return standard deviation,Volume,Mean volume,Standard deviation of volume,Number of transactions
BTCUSDT,93293.869912,6004.338752,-5.570632e-09,0.000016,2.621014e+05,0.007518,0.049750,34861619.0
ETHUSDT,2634.667976,218.946765,-1.209272e-08,0.000075,5.698314e+06,0.177222,0.963579,32153492.0
SOLUSDT,182.473900,26.093761,-5.987317e-08,0.000086,1.314250e+07,1.759114,14.773597,7471089.0
BNBUSDT,626.237857,44.025510,-1.327660e-08,0.000075,3.006684e+06,0.282042,2.367477,10660410.0
AVAXUSDT,25.178913,2.627128,-6.506126e-07,0.000416,6.343330e+06,9.577367,30.454966,662325.0
UNIUSDT,9.027087,0.826547,-2.369548e-07,0.000254,2.510151e+07,13.179367,66.841315,1904607.0
LINKUSDT,18.675309,2.327668,-7.285466e-07,0.000558,1.258030e+07,17.294619,85.752508,727411.0
AXSUSDT,4.139669,0.368972,-9.032063e-07,0.000478,1.115125e+07,27.449358,55.701244,406248.0
RENDERUSDT,4.332628,0.478448,-4.881599e-07,0.000322,2.768562e+07,31.576981,103.133609,876766.0


In [1]:
for name in dir():
    if not name.startswith('_') and name not in ['important_var1', 'important_var2']:
        del globals()[name]
    
import numpy as np
import pandas as pd
from data.DataManager import DataManager
from data.DataGenerator import DataGenerator
from data.DataCollectors import HistoricalDataCollector, HistoricalDataCollectorParquet
from main.RandomnessAnalysis import RandomnessAnalysis
from main.PredictableDayAnalysis import PredictableDayAnalysis
from utils.Analysis import get_assets_properties, localization_predictable_intervals
from utils.MultiTester import MultiTester
from utils.VisualizationTools import plot_block_frequencies
from utils.Analysis import intervals_analysis
from utils.VisualizationTools import plot_test
asset_pairs = ['BTCUSDT','ETHUSDT','SOLUSDT',
               'BNBUSDT', 'AVAXUSDT', 'UNIUSDT',
               'LINKUSDT', 'AXSUSDT', 'RENDERUSDT']
symbols = {
            0: [(-np.inf, 0)
                , (False, False)],
            1: [(0, np.inf), (False, False)]
        }
s = 2
year, month, day = 2025, 2, None